In [ ]:
%%capture
!pip install flask-ngrok

In [ ]:
!pip install flask-bootstrap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460122 sha256=270e7ff4862e0aedece09c2e7bd4657b3cf4e80ae68b961d0f7e455a55e0945e
  Stored in directory: /root/.cache/pip/wheels/6f/33/ad/26540e84a28334e5dfeda756df270f95353779f03bc5cf40d4
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3925 sha256=bab1c54fd27388d6481329a4cf2c323ea3b7b157043a17bc408f84e498e8d46a
  Stored in directory: /root/.cache/pip/wheels/19/31/99/2ec5b4459cac4d801d6201d501a354366d180afc9f8bb2d333
Successfully built flask-bootstrap visitor


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flask import Flask, render_template, request
from flask_bootstrap import Bootstrap
from flask_ngrok import run_with_ngrok

In [ ]:
!pip uninstall -y opencv-python opencv-contrib-python && pip install sleap


Found existing installation: opencv-python 4.7.0.72
Uninstalling opencv-python-4.7.0.72:
  Successfully uninstalled opencv-python-4.7.0.72
Found existing installation: opencv-contrib-python 4.7.0.72
Uninstalling opencv-contrib-python-4.7.0.72:
  Successfully uninstalled opencv-contrib-python-4.7.0.72
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import sleap
import io
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.cm as cm
import cv2
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import numpy as np
import os

In [ ]:
app = Flask(__name__,template_folder='/content/drive/MyDrive/templates')
##app._static_folder = '/content/static'
bootstrap = Bootstrap(app)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    #return '/content/template/index.html'
    return render_template('index2.html')


ALLOWED_EXTENSIONS = ['mp4']

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/upload', methods=['POST'])
def upload():
    if 'video' not in request.files:
        return 'No video file found'
    video = request.files['video']
    if video.filename == '':
        return 'No video selected'
    if video and allowed_file(video.filename):
        #print("ssssssssssssss")
        video.save('/content/sample_data' + video.filename)
        VideoPath = ('/content/sample_data' + video.filename)
        frame_count = get_frame_count(VideoPath) - 1
        !sleap-track $VideoPath \
        --frames 0-$frame_count \
        --tracking.tracker simple \
        -m /content/drive/MyDrive/models/230513_124413.centered_instance \
        -m /content/drive/MyDrive/models/230513_124413.centroid
        PredictFile = ('/content/sample_data' + video.filename + '.predictions.slp')
        print(PredictFile)
        labels = sleap.load_file(PredictFile)
        #print(labels)
        # Here I'm removing the tracks so we just have instances without any tracking applied.
        for instance in labels.instances():
            instance.track = None
        labels.tracks = []
        print(labels)
        tracker = sleap.nn.tracking.Tracker.make_tracker_by_name(
            # General tracking options
            tracker="flow",
            track_window=3,

            # Matching options
            similarity="instance",
            match="greedy",
            min_new_track_points=1,
            min_match_points=1,

            # Optical flow options (only applies to "flow" tracker)
            img_scale=0.5,
            of_window_size=21,
            of_max_levels=3,

            # Pre-tracking filtering options
            target_instance_count=2,
            pre_cull_to_target=True,
            pre_cull_iou_threshold=0.8,

            # Post-tracking filtering options
            post_connect_single_breaks=True,
            clean_instance_count=0,
            clean_iou_threshold=None,
        )
        tracked_lfs = []
        for lf in labels:
            lf.instances = tracker.track(lf.instances, img=lf.image)
            tracked_lfs.append(lf)
        tracked_labels = sleap.Labels(tracked_lfs)
        print(tracked_labels)



        for i in range(0,frame_count):
             tracked_labels[i].plot(scale=1.0)
             plt.savefig(str(i) +'.jpg')
             plt.close()




        image_folder = '/content'
        video_name = '/content/Output.avi'
        images = [img for img in os.listdir(image_folder) if img.endswith('.jpg')]
        #print(images)
        frame = cv2.imread(os.path.join(image_folder, images[0]))
        #print(frame)
        height, width, layers = frame.shape
        video = cv2.VideoWriter(video_name, 0, 1, (width,height))
        #print(video)
        for image in images:
          video.write(cv2.imread(os.path.join(image_folder, image)))

        #cv2.destroyAllWindows()
        video.release()
        #return render_template('preview.html')
    return 'Invalid video file'

#@app.route("/get_frame_count/<video_path>")
def get_frame_count(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-36:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 714, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packag

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Started inference at: 2023-06-25 09:11:25.061519
Args:
{
│   'data_path': '/content/sample_dataEWNIPXGD.mp4',
│   'models': [
│   │   '/content/drive/MyDrive/models/230513_124413.centered_instance',
│   │   '/content/drive/MyDrive/models/230513_124413.centroid'
│   ],
│   'frames': '0-67',
│   'only_labeled_frames': False,
│   'only_suggested_frames': False,
│   'output': None,
│   'no_empty_frames': False,
│   'verbosity': 'rich',
│   'video.dataset': None,
│   'video.input_format': 'channels_last',
│   'video.index': '',
│   'cpu': False,
│   'first_gpu': False,
│   'last_gpu': False,
│   'gpu': 'auto',
│   'max_edge_length_ratio': 0.25,
│   'dist_penalty_weight': 1.0,
│   'batch_size': 4,
│   'open_in_gui': False,
│   'peak_threshold': 0.2,
│   'max_instances': None,
│   'tracking.tracker': 'simple',
│   'tracking.target_instance_count': None,
│   'tracking.pre_cull_to_target': None,
│   'tracking.pre_cull_iou_threshold': None,
│  

INFO:werkzeug:127.0.0.1 - - [25/Jun/2023 09:13:02] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2023 09:13:03] "GET /favicon.ico HTTP/1.1" 404 -


## **Link to open  Flask App**

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://07zt30yjkhre-496ff2e9c6d22116-5000-colab.googleusercontent.com/
